In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [114]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        # convolution
        self.conv1 = nn.Conv2d(1, 4, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(4, 8, 3, padding=1)
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1)
        self.conv4 = nn.Conv2d(16, 24, 3, padding=1)
        self.conv5 = nn.Conv2d(24, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(32, 10)
        self.batchnorm1 = nn.BatchNorm2d(4)
        self.batchnorm2 = nn.BatchNorm2d(8)
        self.batchnorm3 = nn.BatchNorm2d(16)
        self.batchnorm4 = nn.BatchNorm2d(24)
        self.batchnorm5 = nn.BatchNorm2d(32)
        self.dropout1 = nn.Dropout(0.10)
        self.dropout2 = nn.Dropout(0.10)

    def forward(self, x):
        x = self.dropout1(self.pool1(self.batchnorm3(F.relu(self.conv3(self.batchnorm2(F.relu(self.conv2(self.batchnorm1(F.relu(self.conv1(x)))))))))))
        x = self.dropout2(self.pool2(self.batchnorm5(F.relu(self.conv5(self.batchnorm4(F.relu(self.conv4(x))))))))
        x = self.avgpool(x)
        x = x.view(-1, 32)
        x = self.fc1(x)
        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [115]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net2().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
       BatchNorm2d-2            [-1, 4, 28, 28]               8
            Conv2d-3            [-1, 8, 28, 28]             296
       BatchNorm2d-4            [-1, 8, 28, 28]              16
            Conv2d-5           [-1, 16, 28, 28]           1,168
       BatchNorm2d-6           [-1, 16, 28, 28]              32
         MaxPool2d-7           [-1, 16, 14, 14]               0
           Dropout-8           [-1, 16, 14, 14]               0
            Conv2d-9           [-1, 24, 14, 14]           3,480
      BatchNorm2d-10           [-1, 24, 14, 14]              48
           Conv2d-11           [-1, 32, 14, 14]           6,944
      BatchNorm2d-12           [-1, 32, 14, 14]              64
        MaxPool2d-13             [-1, 32, 7, 7]               0
          Dropout-14             [-1, 3

<ipython-input-114-31a9b807852d>:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [116]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [117]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [119]:

model = Net2().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1, verbose=True)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Adjusting learning rate of group 0 to 1.0000e-02.


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-114-31a9b807852d>:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.10238458961248398 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.60it/s]



Test set: Average loss: 0.1319, Accuracy: 9686/10000 (97%)

Adjusting learning rate of group 0 to 1.0000e-02.


loss=0.1472388654947281 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 27.39it/s]



Test set: Average loss: 0.0775, Accuracy: 9784/10000 (98%)

Adjusting learning rate of group 0 to 1.0000e-02.


...


Test set: Average loss: 0.0216, Accuracy: 9930/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-03.


loss=0.004504137672483921 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.12it/s]



Test set: Average loss: 0.0210, Accuracy: 9940/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-03.


loss=0.012561395764350891 batch_id=363:  78%|███████▊  | 364/469 [00:13<00:03, 26.72it/s]


KeyboardInterrupt: 